In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import pickle
import re
import os
import json
from tqdm import tqdm
from tqdm.auto import trange
import arcgis
from arcgis.gis import GIS, ContentManager
from arcgis.mapping import WebMap
from arcgis.geocoding import batch_geocode, get_geocoders
import geopandas as gpd
import pandas as pd
import warnings
import time
import json


class sDataFrame():
    def __init__(self, year = 2019):
        self.year = year
        self._county_gdf = None
        self._tract_gdf = None
        self._tract_gdf_new = None
        
        
    @classmethod
    def read_pickle(cls, file_path):
        """
        read a pickle file and create a class attribute 'data_dictionary'
        ---
        provide a file_path for a pickle file that contains a dictionary of pandas dataframes
        """
        d = pickle.load(open(file_path,'rb'))
        cls.data_dictionary = d

    @classmethod
    def read_csv(cls, file_path, names = None):
        cls.data_dictionary = {}
        if isinstance(file_path, list): # if the user provides a list of files (with or without names of each data)
            if names:
                pass
            else:
                names = range(len(file_path))
            for path, name in zip(file_path, names):
                name = str(name)
                df = pd.read_csv(path)
                if df.columns.str.contains(re.compile('.*tract.*', flags = re.I)).sum():
                    if bool(re.match(re.compile(r'.*_tract', flags = re.I), name)):
                        cls.data_dictionary[name] = df
                    else:
                        name = name + '_tract'
                        cls.data_dictionary[name] = df
                else:
                    if bool(re.match(re.compile(r'.*_county', flags = re.I), name)):
                        cls.data_dictionary[name] = df
                    else:
                        name = name + '_county'
                        cls.data_dictionary[name] = df
        elif isinstance(file_path, str):
            df = pd.read_csv(file_path)
            if names:
                pass
            else:
                if df.columns.str.contains(re.compile('.*tract.*', flags = re.I)).sum():
                    names = 'data_tract'
            cls.data_dictionary[names] = df

    @property
    def countyData(self):
        if self.data_dictionary:
            self._countyData = {k:v for k,v in self.data_dictionary.items() if k[-6:] == 'county'}
            return self._countyData
        else:
            raise AttributeError("data_dictionary has not been created")

            
    @property
    def tractData(self):
        if self.data_dictionary:
            self._tractData  = {k:v for k,v in self.data_dictionary.items() if k[-5:] == 'tract' }
            return self._tractData
        else:
            raise AttributeError("data_dictionary has not been created")
            
    @property
    def pointData(self):
        if self.data_dictionary:
            pat = re.compile('.*(cancer|county|tract)', flags = re.I)
            self._pointData = {k:v for k,v in self.data_dictionary.items() if not bool(re.match(pat, k))}
            return self._pointData
        else:
            raise AttributeError("data_dictionary has not been created")

    @property
    def cancerData(self):
        if self.data_dictionary:
            pat = re.compile('.*(cancer).*', flags = re.I)
            self._pointData = {k:v for k,v in self.data_dictionary.items() if (k[-4:] != 'long') & bool(re.match(pat, k))}
            return self._pointData
        else:
            raise AttributeError("data_dictionary has not been created")

            
    @staticmethod
    def find_state_fips(examples, idx = 0):
        examples = {k: v for k,v in examples.items() if not k[-5:] =='_long' }
        state_fips = examples[list(examples.keys())[idx]].reset_index().FIPS.astype(str).apply(lambda x: x[:2]).unique().tolist()
        return state_fips
    
    @staticmethod
    def find_county_fips(examples, idx = 0):
        county_fips = examples[list(examples.keys())[idx]].reset_index().FIPS.unique().tolist()
        return county_fips

    
    @property
    def state_fips(self):
        if self.data_dictionary:
            self._state_fips = self.find_state_fips(self.data_dictionary)
            return self._state_fips
        else:
            raise AttributeError("data_dictionary has not been created")
            
    @property
    def county_fips(self):
        if self.data_dictionary:
            self._county_fips = self.find_county_fips(self.countyData)
            return self._county_fips
        else:
            raise AttributeError("data_dictionary has not been created")


    @property
    def tiger_census_county(self):
        county = f'https://www2.census.gov/geo/tiger/TIGER{self.year}/COUNTY/tl_{self.year}_us_county.zip'
        gdf = gpd.read_file(county)
        gdf = gdf.loc[gdf.STATEFP.isin(self.state_fips), ['GEOID','geometry']]
        gdf.GEOID = gdf.GEOID.astype(int)
        self._county_gdf =  gdf.reset_index(drop = True)
        return self._county_gdf
        
    @property
    def tiger_census_tract(self):
        states = self.state_fips
        county_fips = self.county_fips
        final_gdf = self.get_tiger_census(self.year, states, county_fips)
        self._tract_gdf = final_gdf
        self._tract_num = final_gdf.GEOID.unique().shape[0]
        return self._tract_gdf
    
    @property
    def next_tiger_census_tract(self):
        states = self.state_fips
        county_fips = self.county_fips
        year = self.year + 1
        final_gdf = self.get_tiger_census(year, states, county_fips)
        self._tract_gdf_new = final_gdf
        self._tract_num_new = final_gdf.GEOID.unique().shape[0]
        return self._tract_gdf_new

    
    @staticmethod
    def get_tiger_census(year, state, county_FIPS = None):
        def transform_gdf(gdf, county_fips):
            county_fips = [str(x) for x in county_fips]
            if county_fips is not None:
                gdf = gdf.loc[countyfp.isin(county_fips), ['GEOID','geometry']]
            else:
                gdf = gdf.loc[:,  ['GEOID','geometry']]
            return gdf
        
        if isinstance(state, int) or isinstance(state, str):
            tract = f"https://www2.census.gov/geo/tiger/TIGER{year}/TRACT/tl_{year}_{state}_tract.zip"
            gdf = gpd.read_file(tract)
            countyfp = gdf.STATEFP + gdf.COUNTYFP
            gdf = transform_gdf(gdf, county_FIPS)
            gdf.GEOID = gdf.GEOID.astype(int)
            final_gdf = gdf.reset_index(drop = True)
            return final_gdf
        elif isinstance(state, list):
            if len(state) == 1:
                state = state[0]
                tract = f"https://www2.census.gov/geo/tiger/TIGER{year}/TRACT/tl_{year}_{state}_tract.zip"
                gdf = gpd.read_file(tract)
                countyfp = gdf.STATEFP + gdf.COUNTYFP
                gdf = transform_gdf(gdf, county_FIPS)
                gdf.GEOID = gdf.GEOID.astype(int)
                final_gdf = gdf.reset_index(drop = True)
                return final_gdf
            elif len(state) > 1:
                dataframes = []
                for s in state:
                    tract = f"https://www2.census.gov/geo/tiger/TIGER{year}/TRACT/tl_{year}_{s}_tract.zip"
                    gdf = gpd.read_file(tract)
                    countyfp = gdf.STATEFP + gdf.COUNTYFP
                    gdf = transform_gdf(gdf, county_FIPS)
                    gdf.GEOID = gdf.GEOID.astype(int)
                    dataframes.append(gdf.reset_index(drop = True))
                final_gdf = pd.concat(dataframes)
                return final_gdf
            else:
                raise TypeError("No element in state")


In [2]:
class CIFTool_AGOL(sDataFrame):
    def __init__(self, gis_address, client_id, folder_name = None):
        self.gis_address = gis_address
        self.client_id = client_id
        self._contentManager = None
        super().__init__()
        self.layers = {}
        self.layers_id = {}
        self.countyLayers = {}
        self.tractLayers = {}
        self.pointLayers = {}
        self.tractLayers_id = {}
        self.countyLayers_id = {}
        self.webmaps = {}
        self.groupLayers = {}
        self.error_count = 0
        self.wait = True
        self.waitTime = 5
        self.set_folder(folder_name = folder_name)

    def wait_AGOL(self, waitTime = None, verbose = True, desc = None):
        if waitTime == None:
            waitTime = self.waitTime
        if self.wait:
            if verbose:
                for p in trange(30, desc = desc):
                    time.sleep(round(waitTime/30,5))
            else:
                time.sleep(waitTime)
        else:
            pass
        
    @staticmethod
    def sociodemographic_colname_update(df):
        df = df.rename(columns = {'Total':'Total Population',
                            '18 to 64' : 'Age 18 to 64'})
        return df
    
    @classmethod
    def from_layers(cls, layers, gis_address, client_id):
        obj = cls(gis_address, client_id)
        contentManager = obj.gis
        obj.layers = {}
        keys = list(layers.keys())
        values = list(layers.values())
        for t1 in trange(len(keys), desc = f"Of {len(keys)} groups"):
            key = keys[t1]
            obj.layers[keys[t1]] = {}
            if len(values[t1]) > 0:
                subkey = list(values[t1].keys())
                subvalues = list(values[t1].values())
                for t2 in trange(len(layers[key]), desc = f"Of {len(layers[key])} tables"):
                    k = subkey[t2]
                    layer_id = subvalues[t2]
#         for key, item in layers.items():
#             obj.layers[key] = {}
#             if len(item) > 0:
#                 for t in trange(len(layers)):
#                     k = list(layers.keys())[t]
#                     layer_id = list(layers.items())[t]
#                 for k, layer_id in layers[key].items():
                    group_lyr =  contentManager.search(layer_id, "Feature Layer")
                    obj.wait_AGOL(1, verbose = False)
                    if len(group_lyr) == 0:
                        raise ValueError("No such group_lyr")
                    else:
                        g_lyr = group_lyr[0]
                        lyr = g_lyr.layers[0]
                        obj.layers[key][k] = lyr
        return obj
        
    @classmethod
    def from_pickle(cls, file_path, gis_address, client_id):
        layers = pickle.load(open(file_path,'rb'))
        obj = cls(gis_address, client_id)
        contentManager = obj.gis
        obj.layers = {}
        keys = list(layers.keys())
        values = list(layers.values())
        for t1 in trange(len(keys), desc = f"Of {len(keys)} groups"):
            key = keys[t1]
            obj.layers[keys[t1]] = {}
            if len(values[t1]) > 0:
                subkey = list(values[t1].keys())
                subvalues = list(values[t1].values())
                for t2 in trange(len(layers[key]), desc = f"Of {len(layers[key])} tables"):
                    k = subkey[t2]
                    layer_id = subvalues[t2]
#         for key, item in layers.items():
#             obj.layers[key] = {}
#             if len(item) > 0:
#                 for k, layer_id in layers[key].items():
                    group_lyr =  contentManager.search(layer_id, "Feature Layer")
                    obj.wait_AGOL(1, verbose = False)
                    if len(group_lyr) == 0:
                        raise ValueError("No such group_lyr")
                    else:
                        flag = True
                        while flag:
                            try:
                                g_lyr = group_lyr[0]
                                lyr = g_lyr.layers[0]
                                obj.layers[key][k] = lyr
                                flag = False
                            except:
                                obj.wait_AGOL(3)
                                obj.error_count += 1
                                if obj.error_count >= 5:
                                    contentManager = obj.gis
                                    obj.error_count = 0
        return obj
            
        
    
    def genTractMaps(layers):
        pass
    
    def genCountyMaps(self):
        pass
    
    
    
    
    @staticmethod
    def manage_popups(wm, level = 'county', title  = None):
        for lyr in wm.layers:
            if title is None:
                lyr.popupInfo.title = lyr.popupInfo.title.replace("_", " ").title()
            elif isinstance(title, str):
                lyr.popupInfo.title = title
            else:
                raise Error("title must be a string")
                
            main_fields = [f.label for f in lyr.popupInfo.fieldInfos if f.fieldName != f.label]
            for field in lyr.popupInfo.fieldInfos:
                if field.label.islower():
                    field.label = field.label.title()
                if level.lower() == 'county':
                    labels_to_include  = ['County','State'] + main_fields
                elif level.lower() == 'tract':
                    labels_to_include = ['Tract','County','State'] + main_fields
                else:
                    raise ValueError('level must be either "county" or "tract"')
                    
                if field.label in ['County','State'] + main_fields:
                    field.visible = True
                else:
                    field.visible = False
        return wm 
    
    
    
    def genWebMapFromGroupLayers(self, level= 'county'):
        if self._contentManager:
            contentManager = self._contentManager
        else:
            contentManager = self.gis
        for flname, fl_dict in self.groupLayers.items():
            # first edit alias of each field
            lyr = fl_dict['Layer Object']
            properties = lyr.properties
            for colname in fl_dict['alias']:
                idx, AGOLName = self.getAGOLFieldName(lyr, colname)
                properties['fields'][idx]['alias'] = colname
            lyr.properties = properties
            time.sleep(.25)
            # next, define a renderer for each and create webmaps
            for colname, title in zip(fl_dict['fields'], fl_dict['alias']):
                renderer = self.renderer_definition(lyr, AGOLName)
                new_renderer = self.add_class_details(renderer)
                flag = True
                while flag:
                    try:
                        wm = WebMap()
                        wm.add_layer(lyr, {'title': f'{flname} : {title}',
                                           'renderer': new_renderer})
                        # wm.add_layer(item, {'title': f'{title} : {key}',
                        #                    'renderer' : new_renderer})
                        self.wait_AGOL(1.5, desc = f'Adding the layer "{flname}:{title}" to the map')
                        wm = self.manage_popups(wm, level = level, title = f"{flname} : {title}")
                        self.wait_AGOL(1.5, desc = f"Managing Popup")
                        webmap_item_properties = {'title': f'{flname} : {title}',
                                                 'snippet':f'{flname} : {title}',
                                                 'tags' : ['Python','Automated']}
                        webmap = wm.save(webmap_item_properties, folder = self.AGOL_folder) # testing folder is temporary set-up
                        self.wait_AGOL(3, desc = f'Creating a WebMap "{flname}:{title}"')
                        searchedWebMap = contentManager.search(f'title:"{flname} : {title}"', item_type = 'Web Map')
                        time.sleep(.5)
                        webmap_s = searchedWebMap[0]
                        webmap_s.share(everyone = True)
                        self.wait_AGOL(.5, desc = f"Chaning sharing option")
                        webmap_id = webmap_s.id
                        self.webmaps[title] = webmap_id
                        flag = False
                    except:
                        time.sleep(2)
                        self.error_count += 1
                        if self.error_count >= 5:
                            contentManager = self.gis
                            self.error_count = 0
        return self.webmaps
            
        
        
    def genWebMap(self, layers, title):
        if self._contentManager:
            contentManager = self._contentManager
        else:
            contentManager = self.gis
        wm = WebMap()
        for key, item in layers.items():
            # step 1. set-up alias for each layer
            idx, AGOLName = self.getAGOLFieldName(item, key)
            item.properties['fields'][idx]['alias'] = key
            time.sleep(.25) # give a time to update the property
            # ste 2. set-up renderer with 5 classes
            renderer = self.renderer_definition(item, AGOLName)
            new_renderer = self.add_class_details(renderer)
            # flag = True
            # while flag:
            #     try:
            #         item.manager.update_definition(new_renderer)
            #         self.wait_AGOL(3, desc = f"Updaing the definition of {title}-{key}')
            #         flag = False
            #     except:
            #         time.sleep(2)
            #         self.error_count += 1
            #         if self.error_count >= 5:
            #             contentManager = self.gis
            #             self.error_count = 0
            # item.properties['drawingInfo']['renderer'] = new_renderer
            flag = True
            while flag:
                try:
                    wm.add_layer(item, {'title': f'{title} : {key}',
                                       'renderer': new_renderer})
                    # wm.add_layer(item, {'title': f'{title} : {key}',
                    #                    'renderer' : new_renderer})
                    self.wait_AGOL(5, desc = f'Adding the layer "{title}:{key}" to the map')
                    flag = False
                except:
                    time.sleep(2)
                    self.error_count += 1
                    if self.error_count >= 5:
                        contentManager = self.gis
                        self.error_count = 0
        return wm

        
    @staticmethod
    def getFieldNames(lyr):
        field_names = [x['name'] for x in lyr.properties.fields]
        return field_names
    
    
    @staticmethod
    def getAGOLFieldName(lyr, FieldName):
        flag = True
        while flag:
            try:
                field_names = [x['name'] for x in lyr.properties.fields]
                flag = False
            except:
                time.sleep(2)
                
        AGOLName = FieldName.lower().replace(' ', '_')[:10]
        index = field_names.index(AGOLName)
        return index, AGOLName
#         lyr.properties['fields'][index]['alias'] = FieldName

    
    @staticmethod
    def renderer_definition(lyr, fieldName):
        definition = {
            'type':'classBreaksDef',
            'classificationField':fieldName,
            'classificationMethod':'esriClassifyNaturalBreaks',
            'breakCount':5,
                }
        flag = True
        while flag:
            try:
                renderer = lyr.generate_renderer(definition)
                flag = False
            except:
                time.sleep(5)
        return renderer
        
    @staticmethod
    def add_class_details(renderer):
        def edit_labels(renderer):
            labels = [x['label'].split(' - ') for x in renderer['classBreakInfos']]
            min_label = labels[0][0]
            max_label = labels[4][1]
            if float(min_label) >= 0 and float(max_label) <= 1:
                new_labels = []
                for label in labels:
                    l = round(float(label[0])*100,2)
                    u = round(float(label[0])*100,2)
                    l = str(l); u = str(u)
                    l += '%'; u += '%'
                    new_labels.append(' - '.join([l, u]))
                return new_labels
            else:
                return None

        
        class1 = [237, 248, 233, 216]
        class2 = [186, 228, 179, 216]
        class3 = [116, 196, 118, 216]
        class4 = [49, 163, 84, 216]
        class5 = [0, 109, 44, 216]

        color_scheme = [class1, class2, class3, class4, class5]

        default_symbol = {
                "type" : "esriSFS", 
                "style" : "esriSFSSolid", 
                "color" : [], 
                'outline': {'type': 'esriSLS', 
                            'style': 'esriSLSSolid', 
                            'color': [52, 52, 52, 255],
                            "width" : 0.4}
                    }

        new_labels = edit_labels(renderer)

        for i, class_info in enumerate(renderer['classBreakInfos']):
            if new_labels:
                class_info['label'] = new_labels[i]
            symbol = default_symbol.copy()
            symbol['color'] = color_scheme[i]
            class_info['symbol'] = symbol

        return renderer    
    
#     def create_webmap(self, title, )
    
        
    def genFeatureLayer(self, sdf, title):
        if self._contentManager:
            contentManager = self._contentManager
        else:
            contentManager = self.gis
        lyrs = contentManager.import_data(sdf,
                                  title = title,
                                 folder = self.AGOL_folder)
        self.wait_AGOL(desc = f"Waiting to Upload {title} layer to AGOL")
        return lyrs
    
    
    @property
    def gis(self):
        warnings.simplefilter('ignore')
        gis = GIS(self.gis_address, client_id = self.client_id) # log-in to the AGOL
        print("Successfully logged in as: " + gis.properties.user.username)
        self._gis = gis
        self._contentManager = ContentManager(gis)
        return self._contentManager
                
    
    def set_folder(self, folder_name):
        try:
            gis = self._gis
        except:
            self.gis
            gis = self._gis
        if folder_name is None:
            import random
            import string
            # printing lowercase
            letters = string.ascii_lowercase
            # Webmaps - UTC timestamp
            folder_name = ''.join(random.choice(letters) for i in range(10))
            gis.content.create_folder(folder=folder_name)
            print(f"folder name is {folder_name}")
            self.AGOL_folder = folder_name

        elif isinstance(folder_name, str):
            x = gis.content.create_folder(folder=folder_name)
            if isintance(x, dict):
                pass
            else:
                print("{folder_name} already exists")
            self.AGOL_folder = folder_name
        else:
            raise Error("folder name has to be in the string format")
            

    
    
    
    def genCountyFL(self, verbose = True):
        data_dictionary = self.countyData
        if verbose:
            keys = list(data_dictionary.keys())
            for t1 in trange(len(keys), desc = f"Generating {len(keys)} county-level feature layers groups"):
                key = keys[t1]
                df = data_dictionary[key]
                self.genAreaSDF4FL(df, key, 'county')
        else:
            keys = list(data_dictionary.keys())
            for key in keys:
                df = data_dictionary[key]
                self.genAreaSDF4FL(df, key, 'county')
            
    def genTractFL(self, verbose = True):
        data_dictionary = self.tractData
        print(data_dictionary.keys())
        if verbose:
            keys = list(data_dictionary.keys())
            for t1 in trange(len(keys), desc = f"Generating {len(keys)} tract-level feature layers groups"):
                key = keys[t1]
                df = data_dictionary[key]
                self.genAreaSDF4FL(df, key, 'tract')
        else:
            keys = list(data_dictionary.keys())
            for key in keys:
                df = data_dictionary[key]
                self.genAreaSDF4FL(df, key, 'tract')
        
    
    def genPointFL(self):
        final_sdf = self.batchgeocode(df = self.pointData['facilities_and_providers'])
        for facility in final_sdf['Type'].unique():
            sdf = final_sdf.loc[final_sdf.Type.eq(facility),:]
            self.genFeatureLayer(sdf, title = f"Facilities and Providers : {facility}")
            self.wait_AGOL(2, desc = f"Uploading Point Feature Layer : {facility}")
        
        # self.genFeatureLayer(final_sdf
    
    def batchgeocode(self,df, address_column_name = "Address"):
#         geocoded_address = geocode(address)
        try:
            self.SuggestedBatchSize
        except:
            self.set_geocoder()
        
        df = df.reset_index(drop = True)
        df['State'] = df[address_column_name].str.extract("\s(\w\w)\s\d\d\d\d\d")
        states = df.State.value_counts().head(len(cif.state_fips)).index.tolist()
        df = df.loc[df.State.isin(states),:]

        address = df[address_column_name]
                
        SBS = self.SuggestedBatchSize
        N = len(address)
        
        if N/SBS == N//SBS:
            numSplits = N/SBS
        else:
            numSplits = N//SBS + 1
        
        SHAPES = []
        for i in range(numSplits):
            start = i*SBS
            end = start + SBS
            sample = address[start:end].tolist()
            feature_set = batch_geocode(sample, as_featureset = True)
            feature_set = feature_set.sdf
            feature_set['ResultID'] = feature_set.ResultID + start
            feature_set.set_index('ResultID', inplace = True)
            shape = feature_set.SHAPE
            SHAPES.append(shape)
        
        shapes = pd.concat(SHAPES)
        df['SHAPE'] = None
        df.loc[df.index.isin(shapes.index), 'SHAPE'] = shapes.sort_index()
        df = df.dropna().reset_index(drop = True)
        df.drop('State', axis = 1, inplace = True)
        return df
    
    def set_geocoder(self):
        try:
            gis = self._gis
        except:
            self.gis
            gis = self._gis
        geocoder = get_geocoders(gis)[0]
        self.SuggestedBatchSize = geocoder.properties.locatorProperties.SuggestedBatchSize

    
    def genAreaSDF4FL_new(self, df, name, level = 'county', tqdm = None, keep_NA = False):
        """
        with a county or tract level dataset, it creates feature layer for each column.
        Then, it uploads each feature layer to AGOL. 
        The name of the Feature Layer will be {name}-{column name in a df}
        """
        if len(df.index.names)>1:
            df.reset_index(inplace = True)
        
        if level == 'county':
            if self._county_gdf is not None:
                gdf = self._county_gdf.copy()
            else:
                gdf = self.tiger_census_county.copy()
        elif level == 'tract':
            if self._tract_gdf is not None:
                gdf = self._tract_gdf.copy()
            else:
                gdf = self.tiger_census_tract.copy()
            if df.FIPS.unique().shape[0] != self._tract_num:
                if self._tract_gdf_new is not None:
                    gdf = self._tract_gdf_new.copy()
                else:
                    gdf = self.next_tiger_census_tract.copy()
                    #####################################################################################
                if df.FIPS.unique().shape[0] != self._tract_num_new: # This needs to be updated later
                    diff1 = -df.FIPS.unique().shape[0] + self._tract_num
                    diff2 = -df.FIPS.unique().shape[0] + self._tract_num_new
                    if diff1>=0:
                        gdf = self._tract_gdf.copy()
                    else:
                        gdf = self._tract_gdf_new.copy()
#                     raise ValueError("Tracts for {self.year} and {self.year +1} are not matching your dataset")
        ###########################3
        else:
            raise ValueError("level has to be either 'county' or 'tract'")
        self.layers[name] = {}
        layers = {}
        layer_ids = {}
        df['FIPS'] = df.FIPS.astype(int)
        if df.columns.str.contains('Total').sum() > 0:
            df = self.sociodemographic_colname_update(df) # to be changed
        if df.columns.str.contains(re.compile('LILAT.*')).sum() > 0:
            df = df.rename(columns = {'LILATracts_Vehicle' : 'LILA Tracts Vehicle'}) ## to be changed  
        sdf = df.merge(gdf, how = 'left', left_on = 'FIPS', right_on = 'GEOID') # we need to pick up from here
        sdf.drop('GEOID', axis = 1, inplace = True) # drop the geoid
        sdf = gpd.GeoDataFrame(sdf, geometry = 'geometry')
        sdf = pd.DataFrame.spatial.from_geodataframe(sdf, column_name = 'geometry')
        self.wait_AGOL(.5, verbose = False)
        geo_pat = re.compile(r'(fips|tract|county|state|geometry|type)', flags = re.I)
        # geo_pat2 = re.compile(r'(fips|tract|county|state|type)', flags = re.I)
        columns     = sdf.columns[~sdf.columns.str.match(geo_pat)].to_list()
        # geo_columns = sdf.columns[sdf.columns.str.match(geo_pat2)].to_list()
        final_sdf = sdf
        if keep_NA:
            pass
        else:
            nulls = -final_sdf.isna().apply(lambda x: True if sum(x) > 0 else False, axis = 1)
            final_sdf = final_sdf.loc[nulls, :]
        self.final_sdf = final_sdf
        ls = None
        while ls is None:
            groupLayer = self.genFeatureLayer(final_sdf, name)
            ls = groupLayer.layers
            if ls is None:
                self.error_count += 1
            if self.error_count >= 5:
                contentManager = self.gis
                self.error_count = 0
        simpleLayer = groupLayer.layers[0]
        self.updateLayerName(simpleLayer, name)
        item_id = simpleLayer.properties.serviceItemId
        fields = [x['name'] for x in simpleLayer.properties.fields if x['name'] not in ['FID','fips','county','state','tract','Shape__Area','Shape__Length']]
        self.groupLayers[name] = {'Layer Object': simpleLayer,
                                  'id' : item_id,
                                  'fields': fields,
                                 'alias': columns}
    
    
    

    def genAreaSDF4FL(self, df, name, level = 'county', tqdm = None, keep_NA = False):
        """
        with a county or tract level dataset, it creates feature layer for each column.
        Then, it uploads each feature layer to AGOL. 
        The name of the Feature Layer will be {name}-{column name in a df}
        """
        if len(df.index.names)>1:
            df.reset_index(inplace = True)
        
        if level == 'county':
            if self._county_gdf is not None:
                gdf = self._county_gdf.copy()
            else:
                gdf = self.tiger_census_county.copy()
        elif level == 'tract':
            if self._tract_gdf is not None:
                gdf = self._tract_gdf.copy()
            else:
                gdf = self.tiger_census_tract.copy()
            if df.FIPS.unique().shape[0] != self._tract_num:
                if self._tract_gdf_new is not None:
                    gdf = self._tract_gdf_new.copy()
                else:
                    gdf = self.next_tiger_census_tract.copy()
                if df.FIPS.unique().shape[0] != self._tract_num_new: # This needs to be updated later
                    diff1 = -df.FIPS.unique().shape[0] + self._tract_num
                    diff2 = -df.FIPS.unique().shape[0] + self._tract_num_new
                    if diff1>=0:
                        gdf = self._tract_gdf.copy()
                    else:
                        gdf = self._tract_gdf_new.copy()
#                     raise ValueError("Tracts for {self.year} and {self.year +1} are not matching your dataset")
        else:
            raise ValueError("level has to be either 'county' or 'tract'")
        self.layers[name] = {}
        layers = {}
        layer_ids = {}
        df['FIPS'] = df.FIPS.astype(int)
        if df.columns.str.contains('Total').sum() > 0:
            df = self.sociodemographic_colname_update(df)
        sdf = df.merge(gdf, how = 'left', left_on = 'FIPS', right_on = 'GEOID') # we need to pick up from here
        sdf.drop('GEOID', axis = 1, inplace = True) # drop the geoid
        sdf = gpd.GeoDataFrame(sdf, geometry = 'geometry')
        sdf = pd.DataFrame.spatial.from_geodataframe(sdf, column_name = 'geometry')
        self.wait_AGOL(.5, verbose = False)
        geo_pat = re.compile(r'(fips|tract|county|state|geometry|type)', flags = re.I)
        geo_pat2 = re.compile(r'(fips|tract|county|state|type)', flags = re.I)
        columns     = sdf.columns[~sdf.columns.str.match(geo_pat)].to_list()
        geo_columns = sdf.columns[sdf.columns.str.match(geo_pat2)].to_list()
        for colname in columns:
            if tqdm is not None:
                tqdm.set_description(f"{name} to AGOL - {colname} in progress")
            selected_columns = geo_columns + [colname, 'geometry']
            final_sdf = sdf[selected_columns]
            if keep_NA:
                final_sdf.loc[final_sdf[colname].isna(), colname] = None
            else:
                final_sdf = final_sdf.loc[final_sdf[colname].notna(), :]
            self.final_sdf = final_sdf

            if name:
                layer_name = name + '-' + colname
            else:
                layer_name = colname
            ls = None
            while ls is None:
                groupLayer = self.genFeatureLayer(final_sdf, layer_name )
                ls = groupLayer.layers
                if ls is None:
                    self.error_count += 1
                if self.error_count >= 5:
                    contentManager = self.gis
                    self.error_count = 0
                
            simpleLayer = groupLayer.layers[0]
            self.updateLayerName(simpleLayer, layer_name)
            item_id = simpleLayer.properties.serviceItemId
            self.layers_id[name][colname] = item_id
            self.layers[name][colname] = simpleLayer
            layers[colname] = simpleLayer
            layer_ids[colname] = item_id
        if level == 'county':
            self.countyLayers[name] = layers
            self.countyLayers_id[name] = layer_ids
        elif level == 'tract':
            self.tractLayers[name] = layers
            self.tractLayers_id[name] = layer_ids
            
        return layers
    
    
    def updateLayerName(self, lyr, name):
        flag = True
        while flag:
            try:
                lyr.manager.update_definition({'name':name})
                self.wait_AGOL(3, desc = f"Updating layer name to '{name}'")
                flag = False
            except:
                self.wait_AGOL(3, desc = "Trying one more time")
                self.error_count += 1
                if self.error_count >= 5:
                    contentManager = self.gis
                    self.error_count = 0
    
    @staticmethod
    def point_renderer(color, outline_color = None, size = 6):
        if outline_color is None:
            outline_color = [0,0,0,255]
        else:
            if isinstance(outline_color, list):
                if len(outline_color) != 4:
                    raise Error("outline_color is a list of 4 integers indicating RGB colors and opacity")
                else:
                    pass
            else:
                raise Error("outline_color is a list of 4 integers indicating RGB colors and opacity")

        renderer = {
                  "type": "simple",
                  "symbol": {
                    "type": "esriSMS",
                    "style": "esriSMSCircle",
                    "color": color,
                    "size": size,
                    "angle": 0,
                    "xoffset": 0,
                    "yoffset": 0,
                    "outline": {
                      "color": outline_color,
                      "width": 0.7
                    }
                  }
        }
        # renderer.symbol.url = ''
        # renderer.symbol.imageData = ''
        renderer = arcgis._impl.common._mixins.PropertyMap(renderer)
        return renderer

                    
                    
                    
                    

    def save_layers(self, file_name, directory = None):
        if len(self.layers) == 0:
            raise Error("There is no layer in this object")
        else:
            if directory == None:
                path1 = os.getcwd()
            if file_name[-7:] != '.pickle':
                file_name += '.pickle'
            with open(os.path.join(path1,file_name), 'wb') as dataset:
                pickle.dump(self.layers_id, dataset, protocol=pickle.HIGHEST_PROTOCOL)
                
    def save_webmaps(self, file_name, directory = None):
        if len(self.layers) == 0:
            raise Error("There is no layer in this object")
        else:
            if directory == None:
                path1 = os.getcwd()
            if file_name[-5:] != '.json':
                file_name += '.json'
            with open(file_name, 'w') as f:
                json.dump(cif.webmaps, f)




In [3]:
import pickle
pickle_file_path = './Dartmouth_catchment_data/dataset.pickle'
client_id= 'uPZiN9HJWneCycx3'
gis_address = "https://ky-cancer.maps.arcgis.com"
cif = CIFTool_AGOL(gis_address, client_id)
cif.read_pickle(pickle_file_path)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ky-cancer.maps.arcgis.com/sharing/rest//oauth2/authorize?response_type=code&client_id=uPZiN9HJWneCycx3&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=LUExvHeKQk0DhYafTJPAXYuAO2RGp6
Enter code obtained on signing in using SAML: ········
Successfully logged in as: lee.park
folder name is ynpbmkgfpe


In [5]:
cif.countyData.keys()

dict_keys(['rf_and_screening_county', 'economy_county', 'ht_county', 'sociodemographics_county', 'environment_county'])

Table Names Conversion

|Old | New|
|:---:|:---:|
|rf_and_screening_county| Screening & Risk Factors (County) |
|economy_county| Economics (County) |
|ht_county | Housing & Transportation (County) |
|sociodemographics_county| Sociodemographics (County) |
|environment_county| Environment (County)  |

In [6]:
cif.tractData.keys()

dict_keys(['rf_and_screening_tract', 'economy_tract', 'ht_tract', 'sociodemographics_tract', 'environment_tract'])

In [10]:
cif.data_dictionary.keys()

dict_keys(['rf_and_screening_county', 'rf_and_screening_county_long', 'rf_and_screening_tract', 'rf_and_screening_tract_long', 'cancer_incidence', 'cancer_incidence_long', 'cancer_mortality', 'cancer_mortality_long', 'economy_county', 'economy_county_long', 'economy_tract', 'economy_tract_long', 'ht_county', 'ht_county_long', 'ht_tract', 'ht_tract_long', 'sociodemographics_county', 'sd_county_long', 'sociodemographics_tract', 'sd_tract_long', 'environment_county', 'environment_county_long', 'environment_tract', 'environment_tract_long', 'broadband_speeds', 'facilities_and_providers'])

In [14]:
# Name for Cancer tables
'Cancer Incidence - Age Adj. : All Site'

'Cancer Incidence - Age Adj. : All Site'

In [19]:
cif.data_dictionary['environment_tract'] # Change "Census_Tract_2019" to FIPS
# Get rid of FIPS

,FIPS,County,State,Census_Tract_2019,LILATracts_Vehicle
0,33001,Belknap County,New Hampshire,33001965100,0
14,33001,Belknap County,New Hampshire,33001966500,0
13,33001,Belknap County,New Hampshire,33001966402,0
11,33001,Belknap County,New Hampshire,33001966200,0
10,33001,Belknap County,New Hampshire,33001966100,0
...,...,...,...,...,...
386,50027,Windsor County,Vermont,50027965300,0
385,50027,Windsor County,Vermont,50027965200,0
384,50027,Windsor County,Vermont,50027965100,0
400,50027,Windsor County,Vermont,50027966700,0


In [20]:
cif.countyData.keys()

dict_keys(['rf_and_screening_county', 'economy_county', 'ht_county', 'sociodemographics_county', 'environment_county'])

In [23]:
cif.countyData['rf_and_screening_county'].head(3)
# BMI_Obese -> Obese (BMI over 30)
# Cancer_Prevalence -> Cancer Prevalence
# Currently_Smoke -> Currently Smoke (Audlt)
# Met_Breast_Screen -> Met Breast Screening Recommendations
# Met_Cervical_Screen -> Had Pap Test in Last 3 Years, Age 21-64
# Met_Colon_Screen -> Met Colorectal Screening Recommendations

,,measure,BMI_Obese,Cancer_Prevalence,Currently_Smoke,Met_Breast_Screen,Met_Cervical_Screen,Met_Colon_Screen
FIPS,County,State,,,,,,
33001,Belknap,NH,0.308,0.091,0.157,0.747,0.860,0.703
33003,Carroll,NH,0.294,0.100,0.156,0.747,0.854,0.707
33005,Cheshire,NH,0.300,0.079,0.164,0.724,0.850,0.687


In [25]:
cif.countyData['economy_county'].head(3)
# Insurance Coverage -> Insured
# Medicaid Enrollment -> Enrolled in Medicaid
# Gini Coefficient   -> Gini Coefficient
# Household Income   -> Household Income ($)
# Annual Labor Force Participation Rate (2015-2019) -> Annual Labor Force Participation (2015-2019) 
# Below Poverty -> Living Below Poverty
# Uninsured     -> Uninsured
# Monthly Unemployment Rate (Jun-22)  -> Monthly Unemployment (***-**)

,FIPS,County,State,Insurance Coverage,Medicaid Enrollment,Gini Coefficient,Household Income,Annual Labor Force Participation Rate (2015-2019),Annual Unemployment Rate (2015-2019),Below Poverty,Uninsured,Monthly Unemployment Rate (Jun-22)
0,33001,Belknap County,New Hampshire,0.933626,0.164852,0.4400,67328.0,0.636900,0.037903,0.063430,0.066374,0.019
1,33003,Carroll County,New Hampshire,0.920831,0.165682,0.4715,66932.0,0.610454,0.055430,0.043994,0.079169,0.020
2,33005,Cheshire County,New Hampshire,0.938260,0.141869,0.4239,64686.0,0.639532,0.038719,0.051770,0.061740,0.023


In [32]:
cif.countyData['ht_county'].head(3)
# Vacancy Rate -> Vacancy Rate
# No Vehicle -> Household without Vehicle Access
# Rent Burden (40% Income) -> High Rent Burden

,FIPS,County,State,Vacancy Rate,No Vehicle,Rent Burden (40% Income)
0,33001,Belknap County,New Hampshire,0.340519,0.012621,0.322812
1,33003,Carroll County,New Hampshire,0.463350,0.013166,0.259421
2,33005,Cheshire County,New Hampshire,0.146728,0.017354,0.251565


In [35]:
cif.countyData['sociodemographics_county'].head(3)
# Total -> Total Population
# Under 18 -> Under 18 Years Old
# 18 to 64 -> Age 18 to 64 Years Old
# Over 64 -> Over 64 Years Old
# Below 9th grade -> Did Not Attend High School
# High School -> Graduated High School
# College -> Graduated College
# Advanced Degree -> Completed Gradaute Degree
# White -> White (Non-Hispanic)
# Black -> Black (Non-Hispanic)
# Asian -> Asian (Non-Hispanic)
# Other_Races -> Other Non-Hispanic Races
# Urban_Percentage -> Urbanized Residence

,FIPS,County,State,Total,Under 18,18 to 64,Over 64,Below 9th grade,High School,College,Advanced Degree,White,Black,Hispanic,Asian,Other_Races,Urban_Percentage
0,33001,Belknap County,New Hampshire,61174,0.184343,0.595220,0.220437,0.015503,0.886085,0.316990,0.132532,0.945647,0.006097,0.017704,0.010184,0.020368,0.3372
1,33003,Carroll County,New Hampshire,48461,0.155465,0.561648,0.282887,0.010966,0.908223,0.369685,0.152392,0.955160,0.005200,0.015311,0.007429,0.016900,0.0980
2,33005,Cheshire County,New Hampshire,76040,0.178827,0.621712,0.199461,0.021967,0.896103,0.337057,0.125154,0.938966,0.007746,0.019358,0.009258,0.024671,0.3498


In [37]:
cif.countyData['environment_county'].head(3)
# PWS_Violations_Since_2016 -> Public Water System Violations Since 2016
# LILATracts_Vehicle -> Tracts that are Food Deserts

,FIPS,County,State,PWS_Violations_Since_2016,LILATracts_Vehicle
0,33001,Belknap County,New Hampshire,5.0,0.000000
4,33003,Carroll County,New Hampshire,7.0,0.321585
5,33005,Cheshire County,New Hampshire,1.0,0.076381


In [40]:
cif.tractData['environment_tract'].head(3)
# LILATracts_Vehicle -> Tracts that are Food Deserts

,FIPS,County,State,Census_Tract_2019,LILATracts_Vehicle
0,33001,Belknap County,New Hampshire,33001965100,0
14,33001,Belknap County,New Hampshire,33001966500,0
13,33001,Belknap County,New Hampshire,33001966402,0


In [43]:
cif.data_dictionary['cancer_incidence'].columns
# All Site -> All Cancer Site
# Bladder -> Baldder
# Brain & ONS -> Brain
# Cervix -> Cervical
# Colon & Rectum -> Colorectal
# Corpus Uteri & Uterus -> Uterine
# Esophagus -> Esophageal
# Female Breast -> Female Breast
# Kidney & Renal Pelvis -> Kidney
# Leukemia -> Leukemia
# Liver & IBD -> Liver
# Lung & Bronchus -> Lung
# Melanoma of the Skin -> Melanoma
# Non-Hodgkin Lymphoma -> Non-Hodgkin Lymphoma
# Oral Cavity & Pharynx -> Head & Neck Cancer
# Ovary -> Ovarian
# Pancreas -> Pancreatic
# Prostate -> Prostate
# Stomach -> Stomach
# Thyroid -> Thyroid

Index(['All Site', 'Bladder', 'Brain & ONS', 'Cervix', 'Colon & Rectum',
       'Corpus Uteri & Uterus, NOS', 'Esophagus', 'Female Breast',
       'Kidney & Renal Pelvis', 'Leukemia', 'Liver & IBD', 'Lung & Bronchus',
       'Melanoma of the Skin', 'Non-Hodgkin Lymphoma', 'Oral Cavity & Pharynx',
       'Ovary', 'Pancreas', 'Prostate', 'Stomach', 'Thyroid'],
      dtype='object', name='Site')

Legend title : {table name} - {variable name}
Pop title : {table name}

In [31]:
cif.tractData['environment_tract'].drop('FIPS', axis = 1, inplace = True)
cif.tractData['environment_tract'].rename(columns = {'Census_Tract_2019':'FIPS'}, inplace = True)

In [32]:
cif.genPointFL()
for name, df in cif.countyData.items():
    cif.genAreaSDF4FL_new(df, name, 'county')
cif.genWebMapFromGroupLayers(level = 'county')
for name, df in cif.tractData.items():
    cif.genAreaSDF4FL_new(df, name, 'tract')
cif.genWebMapFromGroupLayers(level = 'tract')

Waiting to Upload Facilities and Providers : Colon & Rectal Surgeon layer to AGOL:   0%|          | 0/30 [00:0…

Uploading Point Feature Layer : Colon & Rectal Surgeon:   0%|          | 0/30 [00:00<?, ?it/s]

Waiting to Upload Facilities and Providers : FQHC layer to AGOL:   0%|          | 0/30 [00:00<?, ?it/s]

Uploading Point Feature Layer : FQHC:   0%|          | 0/30 [00:00<?, ?it/s]

Waiting to Upload Facilities and Providers : Gastroenterology layer to AGOL:   0%|          | 0/30 [00:00<?, ?…

...

Chaning sharing option:   0%|          | 0/30 [00:00<?, ?it/s]

Adding the layer "sociodemographics_county:Under 18" to the map:   0%|          | 0/30 [00:00<?, ?it/s]

Managing Popup:   0%|          | 0/30 [00:00<?, ?it/s]

Creating a WebMap "sociodemographics_county:Under 18":   0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
## So,,, we need to separate county and tract level layers when they are created

In [ ]:
cif.save_layers('AllLayers')

In [ ]:
cif.save_webmaps('AllMaps')

In [ ]:
lyr = cif.groupLayers['rf_and_screening_county']['Layer Object']

In [ ]:
extent = lyr.properties.extent

# Point Variables

In [ ]:
from arcgis.geocoding import batch_geocode

In [ ]:
gis = GIS(gis_address, client_id= client_id) # log-in to the AGOL
print("Successfully logged in as: " + gis.properties.user.username)

In [ ]:
contentManager = ContentManager(gis)

In [ ]:
geocoder = get_geocoders(gis)[0]
print(" - SuggestedBatchSize: " + str(geocoder.properties.locatorProperties.SuggestedBatchSize))


In [ ]:
fac = cif.pointData['facilities_and_providers'].copy()

In [ ]:
fac.reset_index(drop = True, inplace = True)

In [ ]:
fac= fac.loc[fac.Address.notna(), :]

In [ ]:
fac.Phone_number = fac.Phone_number.fillna('NA')

In [ ]:
N = fac.shape[0]

In [ ]:
if N/150 == N//150:
    epoch = N/150
else:
    epoch = N//150 + 1
epoch

In [ ]:
fac['State'] = fac.Address.str.extract("\s(\w\w)\s\d\d\d\d\d")

In [ ]:
states = fac.State.value_counts().head(len(cif.state_fips)).index.tolist()
fac = fac.loc[fac.State.isin(states),:]
fac

In [ ]:
SHAPES = []
for i in range(epoch):
    start = i*150
    end = (i+1)*150
    sample = fac.loc[start:end,'Address'].tolist()
    feature_set = batch_geocode(sample, as_featureset = True)
                               # search_extent = [(ymin, xmin), (ymin, xmax), (ymax, xmin), (ymax, xmax)],
    feature_set = feature_set.sdf
    feature_set['ResultID'] = feature_set.ResultID + start
    feature_set.set_index('ResultID', inplace = True)
    shape  = feature_set.SHAPE
    SHAPES.append(shape)

In [ ]:
shapes = pd.concat(SHAPES)

In [ ]:
fac['SHAPE'] = None
fac

In [ ]:
fac.loc[fac.index.isin(shapes.index),'SHAPE'] = shapes.sort_index()

In [ ]:
fac = fac.dropna().reset_index(drop = True)

In [ ]:
fac.drop('state', axis = 1, inplace  = True)

In [ ]:
lyrs = contentManager.import_data(fac,
                          title = 'point',
                         folder = cif.AGOL_folder)

In [ ]:
lyrs

In [ ]:
lyr = lyrs.layers[0]

In [ ]:
m = WebMap()

In [ ]:
fac.type.unique()

In [1]:
gis = GIS(gis_address, client_id= client_id) # log-in to the AGOL
print("Successfully logged in as: " + gis.properties.user.username)

NameError: name 'GIS' is not defined

In [ ]:
contentManager = ContentManager(gis)

In [ ]:
facilities = contentManager.search('6422a55375a74633a5db50629bf56a47')[0]

In [ ]:
org = facilities.layers[1]

In [ ]:
org.properties.drawingInfo.renderer

In [ ]:
def point_renderer(lyr, color, size = 6):
    renderer = {
              "type": "simple",
              "symbol": {
                "type": "esriSMS",
                "style": "esriSMSCircle",
                "color": color,
                "size": size,
                "angle": 0,
                "xoffset": 0,
                "yoffset": 0,
                "outline": {
                  "color": [
                    0,
                    0,
                    0,
                    255
                  ],
                  "width": 0.7
                }
              }
    }
    # renderer.symbol.url = ''
    # renderer.symbol.imageData = ''
    import arcgis
    renderer = arcgis._impl.common._mixins.PropertyMap(renderer)
    return renderer
# lyr.properties.drawingInfo.renderer.symbol

In [ ]:
import arcgis

In [ ]:
lyr.properties.drawingInfo.renderer

In [ ]:
renderer = update_point_renderer(lyr, [200,200,200,255])

In [ ]:
lyr.query(where = "type='Mammography'").sdf

In [ ]:
fac.phone_number = fac.phone_number.fillna('NA')

In [ ]:
lyrs = contentManager.import_data(fac,
                          title = 'point',
                         folder = cif.AGOL_folder)


In [ ]:
?fac.spatial.to_featurelayer

## What to do next time

* update genSDF4FL function
* make sure layers are named properly


In [ ]:
if self._county_gdf:
    gdf = self._county_gdf.copy()
else:
    gdf = self.tiger_census_county.copy()
countyData = self.countyData
layers = {}
for k in list(countyData.keys()):
    layers[k] = []
    m = countyData[k].copy()
    m['FIPS'] = m.FIPS.astype(int)
    sdf = m.merge(gdf, how = 'left', left_on = 'FIPS', right_on = 'GEOID') # we need to pick up from here
    sdf.drop('GEOID', axis = 1, inplace = True) # drop the geoid
    sdf = gpd.GeoDataFrame(sdf, geometry = 'geometry')
    sdf = pd.DataFrame.spatial.from_geodataframe(sdf, column_name = 'geometry')
    geo_pat = re.compile('(fips|tract|county|state|geometry|type)', flags = re.I)
    geo_pat2 = re.compile('(fips|tract|county|state|type)', flags = re.I)

    columns     = sdf.columns[~sdf.columns.str.contains(geo_pat)].to_list()
    geo_columns = sdf.columns[sdf.columns.str.contains(geo_pat2)].to_list()
    for c in columns:
        selected_columns = geo_columns + [c, 'geometry']
        final_sdf = sdf[selected_columns]
        lyr = genFeatureLayer(final_sdf, k + '-' + c)
        layers.append(lyr)


In [ ]:
geo_pat = re.compile('(fips|tract|county|state|geometry)', flags = re.I)
sdf.columns[~sdf.columns.str.contains(geo_pat)]

In [ ]:
c.tractData['economy_tract']

In [ ]:
c.pointData.keys()

In [ ]:
c.cancerData.keys()

In [ ]:
c.data_dictionary.keys()

In [ ]:
c.data_dictionary['cancer_incidence'].index.names

In [ ]:
c.tiger_census_tract

In [ ]:
c.cancerData['cancer_incidence']

In [ ]:
c.tiger_census_county

In [ ]:
c._county_gdf